## **Install Important Libraries**

In [4]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/fb/c4/f574ba6f04e6d7bf8c38d23e7a52389566dd7631fee0bcdd79ea07ef2dbf/opencv_python-4.8.0.76-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 544.7 kB/s eta 0:01:10
   ---------------------------------------- 0.1/38.1 MB 655.4 kB/s eta 0:00:58
   ---------------------------------------- 0.1/38.1 MB 599.1 kB/s eta 0:01:04
   ---------------------------------------- 0.1/38.1 MB 607.9 kB/s eta 0:01:03
   ---------------------------------------- 0.2/38.1 MB 697.2 kB/s eta 0:00:55
   ---------------------------------------- 0.2/38.1 MB 654.4 kB/s eta 0:00:58
   ---------------------------------------- 0.2/38.1 MB 653.6 kB/s eta 0:00:58
   -------------

In [6]:
!pip install tensorflow

## **Import Important Libraries**

In [1]:
import numpy as np #for linear algebra
import cv2 #for real-time optimized Computer Vision models
import tensorflow #to use deep neural networks
from keras.models import Sequential #to use the sequential model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D #for model layers
from keras.optimizers import Adam #for optimizing 
from keras.preprocessing.image import ImageDataGenerator #for data augmentation


import tkinter as tk
from tkinter import *
from PIL import Image, ImageTk
import os

## **Read the dataset files**

In [2]:
#read the training data file
train_directory = 'emoji/train'
#read the testing data file
test_directory = 'emoji/test'

## **Prepare the data for modeling process**

In [10]:
#rescale the training data
train_data_generator = ImageDataGenerator(rescale=1./255)
#rescale the testing data
test_data_generator = ImageDataGenerator(rescale=1./255)

In [12]:
#apply data augmentation on the training data with size of 48*48 and grayscale mode
train_generator = train_data_generator.flow_from_directory(
        train_directory,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [13]:
#apply data augmentation on the testing data with size of 48*48 and grayscale mode
validation_generator = test_data_generator.flow_from_directory(
        test_directory,
        target_size=(48,48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 7178 images belonging to 7 classes.


## **Define the sequential model**

In [14]:
model = Sequential()

#add conv2D layer for spatial convolution over images
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48,48,1)))
model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
#add MaxPooling2D to Downsample the input along its spatial dimensions
model.add(MaxPooling2D(pool_size=(2, 2)))
#add Dropout layer to prevent overfitting
model.add(Dropout(0.25))

#add conv2D layer for spatial convolution over images
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
#add MaxPooling2D to Downsample the input along its spatial dimensions
model.add(MaxPooling2D(pool_size=(2, 2)))
#add conv2D layer for spatial convolution over images
model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
#add MaxPooling2D to Downsample the input along its spatial dimensions
model.add(MaxPooling2D(pool_size=(2, 2)))
#add Dropout layer to prevent overfitting
model.add(Dropout(0.25))

model.add(Flatten())
#to use Rectified Linear Unit activation function.
model.add(Dense(1024, activation='relu'))
#add Dropout layer to prevent overfitting
model.add(Dropout(0.5))
#to define the Softmaxed output with the same shape as inputs.
model.add(Dense(7, activation='softmax'))

## **Compile the model**

In [19]:
#compile the sequential model with categorical_crossentropy and Adam optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=tensorflow.keras.optimizers.legacy.Adam(lr=0.0001, decay=1e-6),
              metrics=['accuracy'])

D:\Anaconda\Lib\site-packages\keras\src\optimizers\legacy\adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


## **Fit the model on the images data**

In [21]:
#fit the model with the training augmented data with 50 epochs
model.fit_generator(
        train_generator,
        steps_per_epoch = 28709 // 64,
        epochs=50,
        validation_data=validation_generator,
        validation_steps = 7178 // 64)

C:\Users\HP\AppData\Local\Temp\ipykernel_13336\1808512500.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
448/448 [==============================] - 244s 541ms/step - loss: 1.8087 - accuracy: 0.2555 - val_loss: 1.7409 - val_accuracy: 0.3306
Epoch 2/50
448/448 [==============================] - 285s 637ms/step - loss: 1.6414 - accuracy: 0.3613 - val_loss: 1.5550 - val_accuracy: 0.4044
Epoch 3/50
448/448 [==============================] - 284s 634ms/step - loss: 1.5429 - accuracy: 0.4076 - val_loss: 1.4752 - val_accuracy: 0.4375
Epoch 4/50
448/448 [==============================] - 286s 638ms/step - loss: 1.4694 - accuracy: 0.4380 - val_loss: 1.4095 - val_accuracy: 0.4602
Epoch 5/50
448/448 [==============================] - 287s 642ms/step - loss: 1.4027 - accuracy: 0.4659 - val_loss: 1.3699 - val_accuracy: 0.4814
Epoch 6/50
448/448 [==============================] - 274s 612ms/step - loss: 1.3528 - accuracy: 0.4854 - val_loss: 1.3093 - val_accuracy: 0.5028
Epoch 7/50
448/448 [==============================] - 268s 599ms/step - loss: 1.3034 - accuracy: 0.5049 - val_loss: 1.2804 -

In [22]:
#save the model weights
model.save_weights('model.h5')